# Final algorithm

Mock desing in python to test the idea. This is the final one

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import sys
from pyEliasFano import EliasFano, UniformlyPartitionedEliasFano

In [2]:
MIN_PER_RANK = 1  # Nodes/Rank: How 'fat' the DAG should be.
MAX_PER_RANK = 5
MIN_RANKS = 3     # Ranks: How 'tall' the DAG should be.
MAX_RANKS = 5
PERCENT = 30      # Chance of having an Edge.

In [3]:
def generate_dag(min_w, max_w, total_nodes): # min_w, max_w: node values range, total_nodes: total number of nodes
    random.seed()  # Initialize the random number generator

    G = nx.DiGraph()

    current_nodes = 0 # Total number of nodes in the graph
    ranks = [] # Number of nodes in each rank

    # Generate ranks with nodes until the total number of nodes is reached
    while current_nodes < total_nodes:
        new_nodes = min(MAX_PER_RANK, total_nodes - current_nodes) # type: ignore # Number of nodes in the new rank
        ranks.append(new_nodes) # Add the new rank to the list of ranks
        current_nodes += new_nodes # Update the total number of nodes

    nodes = 1 # Total number of nodes in the graph starts from 1

    for rank in ranks:
        for k in range(rank):
            # Assign a random weight to each new node
            node_weight = random.randint(min_w, max_w)
            G.add_node(nodes + k, weight=node_weight)

        # Edges from old nodes ('nodes') to new ones ('rank').
        for j in range(nodes - 1): # Adjusted to start from 0
            for k in range(rank):
                if random.randint(0, 99) < PERCENT: # type: ignore # Randomly decide if there is an edge between the nodes
                    G.add_edge(j + 1, k + nodes) # Adjusted to start from 1

        nodes += rank  # Accumulate into old node set.

    # remove isolated nodes
    G.remove_nodes_from(list(nx.isolates(G)))

    root_id = 0 # Root node is 0

    roots = [node for node in G.nodes() if G.in_degree(node) == 0] # Find the root nodes
    for _ in roots:
        G.add_edge(root_id, _) # Add an edge from the root node to each root node

    # add an attribute 'out' to the root node as a set containing 0
    G.nodes[root_id]['out'] = []
    G.nodes[root_id]['out'].append(0)
    G.nodes[root_id]['weight'] = 0

    return G

def compute_node_out(G, node_id):
    node_weight = G.nodes[node_id]['weight']
    predecessors = list(G.predecessors(node_id))

    node_out = []
    for predecessor in predecessors:
        predecessor_out = G.nodes[predecessor]['out']
        # Extend the list instead of union
        node_out.extend(predecessor_out)

    # Ensure uniqueness and sort
    node_out = sorted(set([x + node_weight for x in node_out]))
    return node_out

def optimal_father(G, node_id):
    G.nodes[node_id]['optimal_father'] = None
    successors = list(G.successors(node_id))
    if len(successors) == 0:
        return

    max_out = -1
    for successor in successors:
        successor_out = G.nodes[successor]['out']
        if len(successor_out) > max_out:
            max_out = len(successor_out)
            G.nodes[node_id]['optimal_father'] = successor

def offset(G, node_id):
    optimal_father = G.nodes[node_id]['optimal_father']
    father_weight = G.nodes[optimal_father]['weight']
    father_out = G.nodes[optimal_father]['out']
    node_out = G.nodes[node_id]['out']

    offset = [] #list of indexes of the father_out that are in node_out
    for i in range(len(father_out)):
        if father_out[i] - father_weight in node_out:
            offset.append(i)

    return offset

def process_graph(G):
    for node in list(nx.topological_sort(G))[1::]: # the root is already initialized with (0)
        G.nodes[node]['out'] = compute_node_out(G, node)
    for node in G.nodes():
        optimal_father(G, node)
    for node in G.nodes():
        if G.nodes[node]['optimal_father'] is not None:
            G.nodes[node]['offset'] = offset(G, node)
        else:
            G.nodes[node]['offset'] = None

In [4]:
def compressed_graph(G) -> nx.DiGraph:
    G_compressed = G.copy()
    for node in G_compressed.nodes():
            if G_compressed.nodes[node]['offset'] is not None:
                offset = G_compressed.nodes[node]['offset']
            ef0 = EliasFano(offset)
            G_compressed.nodes[node]['offset'] = ef0
    return G_compressed

In [5]:
G = generate_dag(1, 10, 30)
process_graph(G)
for node in G.nodes():
    print(f"Node {node}: {G.nodes[node]}")

Node 1: {'weight': 9, 'out': [9], 'optimal_father': 29, 'offset': [1]}
Node 2: {'weight': 9, 'out': [9], 'optimal_father': 26, 'offset': [3]}
Node 3: {'weight': 3, 'out': [3], 'optimal_father': 26, 'offset': [0]}
Node 4: {'weight': 3, 'out': [3], 'optimal_father': 28, 'offset': [0]}
Node 5: {'weight': 7, 'out': [7], 'optimal_father': 26, 'offset': [1]}
Node 6: {'weight': 1, 'out': [4], 'optimal_father': 25, 'offset': [0]}
Node 7: {'weight': 5, 'out': [8], 'optimal_father': 17, 'offset': [1]}
Node 8: {'weight': 5, 'out': [14], 'optimal_father': 28, 'offset': [4]}
Node 9: {'weight': 10, 'out': [13, 19], 'optimal_father': 29, 'offset': [4, 9]}
Node 10: {'weight': 2, 'out': [9], 'optimal_father': 29, 'offset': [1]}
Node 11: {'weight': 7, 'out': [10, 16], 'optimal_father': 24, 'offset': [2, 6]}
Node 12: {'weight': 6, 'out': [19, 25], 'optimal_father': 29, 'offset': [9, 15]}
Node 13: {'weight': 10, 'out': [13, 14, 17, 18, 23, 29], 'optimal_father': 17, 'offset': [4, 5, 7, 8, 9, 10]}
Node 14:

In [6]:
G_compressed = compressed_graph(G)
for node in G_compressed.nodes():
    print(f"Node {node}: {G_compressed.nodes[node]}")

Node 1: {'weight': 9, 'out': [9], 'optimal_father': 29, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d549a0>}
Node 2: {'weight': 9, 'out': [9], 'optimal_father': 26, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d55870>}
Node 3: {'weight': 3, 'out': [3], 'optimal_father': 26, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d54940>}
Node 4: {'weight': 3, 'out': [3], 'optimal_father': 28, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d54cd0>}
Node 5: {'weight': 7, 'out': [7], 'optimal_father': 26, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d552d0>}
Node 6: {'weight': 1, 'out': [4], 'optimal_father': 25, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d55000>}
Node 7: {'weight': 5, 'out': [8], 'optimal_father': 17, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690d54910>}
Node 8: {'weight': 5, 'out': [14], 'optimal_father': 28, 'offset': <pyEliasFano.EliasFano.EliasFano object at 0x79e690

# Move from networkx to a custom class to improve performance

In [7]:
class Node:
    def __init__(self, node_id, weight, successor_id=None, offset=None):
        self.node_id = node_id
        self.weight = weight
        self.successor_id = successor_id
        self.offset = offset

    def __repr__(self):
        return (f"Node(id={self.node_id}, weight={self.weight}, "
                f"successor_id={self.successor_id}, offset={self.offset})")

class DAG:
    def __init__(self):
        self.nodes = {}

    def add_node(self, node_id, weight, successor_id=None, offset=None):
        if node_id in self.nodes:
            raise ValueError(f"Node with id {node_id} already exists.")
        new_node = Node(node_id, weight, successor_id, offset)
        self.nodes[node_id] = new_node

    def remove_node(self, node_id):
        if node_id not in self.nodes:
            raise ValueError(f"Node with id {node_id} does not exist.")
        del self.nodes[node_id]

    def get_node(self, node_id):
        return self.nodes.get(node_id)

    def __repr__(self):
        return '\n'.join(str(node) for node in self.nodes.values())

    # Query functions
    def query(self, node_id, index):
        node = self.get_node(node_id)
        if node is None:
            raise ValueError(f"Node with id {node_id} does not exist.")
        if node.successor_id is None:
            return node.offset.select(index) 

        sum_weights = 0
        current_node = node
        while current_node.successor_id is not None:
            index = current_node.offset.select(index)
            current_node = self.get_node(current_node.successor_id)
            sum_weights += current_node.weight
        return current_node.offset.select(index) - sum_weights  # Use current_node directly

In [8]:
dag = DAG()
for node in G_compressed.nodes():
    # if the node has a successor, then do not represent its explicit set out, but just the offset
    if G_compressed.nodes[node]['optimal_father'] is not None:
        dag.add_node(node, G_compressed.nodes[node]['weight'], G_compressed.nodes[node]['optimal_father'], G_compressed.nodes[node]['offset'])
    # if the node has no successor, then represent its explicit set out
    else:
        out = G_compressed.nodes[node]['out']
        ef0 = EliasFano(out)
        dag.add_node(node, G_compressed.nodes[node]['weight'], None, ef0)

print(dag)

Node(id=1, weight=9, successor_id=29, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d549a0>)
Node(id=2, weight=9, successor_id=26, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d55870>)
Node(id=3, weight=3, successor_id=26, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d54940>)
Node(id=4, weight=3, successor_id=28, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d54cd0>)
Node(id=5, weight=7, successor_id=26, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d552d0>)
Node(id=6, weight=1, successor_id=25, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d55000>)
Node(id=7, weight=5, successor_id=17, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d54910>)
Node(id=8, weight=5, successor_id=28, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d54a30>)
Node(id=9, weight=10, successor_id=29, offset=<pyEliasFano.EliasFano.EliasFano object at 0x79e690d54df0>)
Node(id=10, weight=2, successor_id=29, offset=<pyElias